In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split # Import train_test_split
from sklearn.preprocessing import StandardScaler
from PIL import Image
from tqdm import tqdm
import warnings

# Ensure PIL version compatibility (for resize)
try:
    # PIL 9.0.0+
    LANCZOS_RESAMPLE = Image.Resampling.LANCZOS
except AttributeError:
    # Older PIL
    LANCZOS_RESAMPLE = Image.LANCZOS

# --- Configuration Area ---

# *** 1. Paths have been modified according to your requirements ***
DATA_ROOT = "/mnt/hdd/jiazy/CelebA"
ATTR_FILE = os.path.join(DATA_ROOT, "list_attr_celeba.csv")
BASE_IMAGE_DIR = os.path.join(DATA_ROOT, "img_align_celeba")

# Output directory for processed files
OUTPUT_DIR = "./features"

# --- Key Changes ---
# Target label
LABEL_COLUMN = 'Attractive' 
# New: Image size
IMAGE_SIZE = 224

# --------------------

def preprocess_celeba_flow(attr_file, base_image_dir, output_dir, label_column):
    """
    Preprocess CelebA according to new user requirements (8:1:1 split, CSV loading, 224x224 resize)
    """
    print(f"Starting CelebA preprocessing pipeline (Label: {label_column})...")
    os.makedirs(output_dir, exist_ok=True)
    
    # --- 1. 📖 Load data (Step 1) ---
    # *** 2. Modified: Load CSV instead of TXT, and no longer load partition file ***
    print(" Loading annotation file (list_attr_celeba.csv)...")
    try:
        # Load .csv file
        df_master = pd.read_csv(ATTR_FILE)
    except FileNotFoundError as e:
        print(f"Error: File not found {e.filename}.")
        print("Please ensure DATA_ROOT and ATTR_FILE are set correctly.")
        return
    
    # The partition and merge parts of the original script have been removed
    print(f" > Successfully loaded {len(df_master)} records.")

    # --- 2. 📝 Define features (Step 2) ---
    print(" Defining features...")
    
    # *** 3. Fixed: Initialize as empty list ***
    continuous_cols = []
    
    # Categorical features are 40 attributes
    # We exclude 'image_id'
    # (Assume 'image_id' is the first column of .csv)
    categorical_cols = df_master.columns.drop(['image_id']).tolist()
    
    # Ensure the label we selected is in the categorical feature list (will be removed later)
    if label_column not in categorical_cols:
        print(f"Error: Label '{label_column}' is not in the attribute list.")
        print(f"Available attributes: {categorical_cols}")
        return
        
    # Define non-feature columns
    # *** 4. Modified: Removed 'split' ***
    non_feature_cols = ['image_id', label_column]
    
    print(f" > Defined {len(continuous_cols)} continuous features.")
    print(f" > Defined {len(categorical_cols)} categorical features (including label).")
    print(f" > Target label (Label): {label_column}")

    # --- 8. 🔪 Dataset splitting (Step 8) ---
    # *** 5. Completely modified: Use 8:1:1 stratified split, replacing original fixed split ***
    print(f" Performing 8:1:1 stratified split on {label_column}...")
    
    # First, stratified split to get 80% training set
    train_df, temp_df = train_test_split(
        df_master,
        test_size=0.2,
        stratify=df_master[label_column],
        random_state=42
    )
    
    # Then, split the remaining 20% equally into 10% validation set and 10% test set
    val_df, test_df = train_test_split(
        temp_df,
        test_size=0.5,
        stratify=temp_df[label_column],
        random_state=42
    )
    
    print(f" > Training set: {len(train_df)} samples")
    print(f" > Validation set: {len(val_df)} samples")
    print(f" > Test set: {len(test_df)} samples")

    # --- 3. 🧹 Feature filtering (Step 3) ---
    # (This step logic still applies to the new split)
    print(" Filtering constant features...")
    
    final_continuous_cols = list(continuous_cols)
    final_categorical_cols = list(categorical_cols)
    
    # (This part of code unchanged, but now applies to newly split dataset)
    for col in continuous_cols:
        is_constant_train = train_df[col].nunique() <= 1
        is_constant_test = test_df[col].nunique() <= 1
        if is_constant_train and is_constant_test:
            print(f" > Removing constant continuous feature: {col}")
            final_continuous_cols.remove(col)
            
    for col in categorical_cols:
        is_constant_train = train_df[col].nunique() <= 1
        is_constant_test = test_df[col].nunique() <= 1
        if is_constant_train and is_constant_test:
            print(f" > Removing constant categorical feature: {col}")
            final_categorical_cols.remove(col)
            
    # Ensure target label(label_column) is not treated as a feature
    if label_column in final_categorical_cols:
        final_categorical_cols.remove(label_column)
        print(f" > Isolated '{label_column}' as label, not as feature.")
        
    print(f" > Remaining {len(final_continuous_cols)} continuous features.")
    print(f" > Remaining {len(final_categorical_cols)} categorical features (tabular attributes).")

    # --- 4. 🗺️ Path processing (Step 4) ---
    print(" Converting image paths to absolute paths...")
    
    def create_absolute_path(image_id):
        # Assume image_id is already a filename like '000001.jpg'
        return os.path.join(base_image_dir, image_id)

    train_df['absolute_image_path'] = train_df['image_id'].apply(create_absolute_path)
    val_df['absolute_image_path'] = val_df['image_id'].apply(create_absolute_path)
    test_df['absolute_image_path'] = test_df['image_id'].apply(create_absolute_path)
    
    non_feature_cols.extend(['absolute_image_path'])

    # --- 5. 🗑️ Data cleaning (Step 5) ---
    print(" Filtering invalid rows (based on paths and labels)...")
    initial_train, initial_val, initial_test = len(train_df), len(val_df), len(test_df)
    
    train_df = train_df.dropna(subset=['absolute_image_path', label_column])
    val_df = val_df.dropna(subset=['absolute_image_path', label_column])
    test_df = test_df.dropna(subset=['absolute_image_path', label_column])
    
    print(f" > Training set removed {initial_train - len(train_df)} rows")
    print(f" > Validation set removed {initial_val - len(val_df)} rows")
    print(f" > Test set removed {initial_test - len(test_df)} rows")

    # --- 6. ✏️ Fill missing values (Step 6) ---
    print(" Processing missing values/special encoding...")

    # (Continuous feature part skipped)
    for col in final_continuous_cols:
        mean_val = train_df[col].mean()
        train_df[col] = train_df[col].fillna(mean_val)
        val_df[col] = val_df[col].fillna(mean_val)
        test_df[col] = test_df[col].fillna(mean_val)
        
    # **CelebA specific step**:
    # *** 6. Assumption: We assume .csv still uses -1/1 encoding ***
    # (If your .csv is already 0/1, you can comment out the .replace(-1, 0) line)
    print(" > [CelebA specific] Converting attributes: Map -1 to 0...")
    all_attr_cols = final_categorical_cols + [label_column]
    for col in all_attr_cols:
        train_df[col] = train_df[col].replace(-1, 0)
        val_df[col] = val_df[col].replace(-1, 0)
        test_df[col] = test_df[col].replace(-1, 0)
        # No missing values, no need to process
        
    # --- 7. 🔢 Feature engineering (categorical) (Step 7) ---
    print(" Processing categorical features...")
    
    # *** 7. Fixed: Initialize as empty list ***
    field_lengths = []
    
    print(" > CelebA attributes are already 0/1 encoded, skipping .cat.codes.")
    print(" > Calculating cardinalities...")

    for col in final_categorical_cols:
        all_values = pd.concat([train_df[col], val_df[col], test_df[col]])
        cardinality = all_values.nunique()
        # Add a check in case 'MISSING' was actually filled
        if "MISSING" in all_values.unique():
            print(f" > Warning: Feature {col} contains 'MISSING' values.")
        field_lengths.append(cardinality)
        
    print(f" > Cardinalities of {len(final_categorical_cols)} categorical features (partial): {field_lengths[:5]}...")
    
    # --- 9. 🔢 Feature engineering (continuous) (Step 9) ---
    print(" Standardizing continuous features...")

    for _ in final_continuous_cols:
        field_lengths.insert(0, 1) 
    
    scaler = StandardScaler()
    
    if final_continuous_cols:
        scaler.fit(train_df[final_continuous_cols])
        train_df[final_continuous_cols] = scaler.transform(train_df[final_continuous_cols])
        val_df[final_continuous_cols] = scaler.transform(val_df[final_continuous_cols])
        test_df[final_continuous_cols] = scaler.transform(test_df[final_continuous_cols])
        print(" > Continuous features standardized.")
    else:
        print(" > No continuous features, skipping standardization.")

    # --- 12. 🖼️ Image processing (JPEG to NPY) (Step 12) ---
    # *** 8. Modified: Added 224x224 Resize and fixed npy_path Bug ***
    print(f" Converting images (JPG -> NPY, and Resize to {IMAGE_SIZE}x{IMAGE_SIZE})...")

    def convert_jpg_to_npy(jpg_path):
        """
        Open JPG, Resize, convert to NumPy array, save as .npy, and return .npy path
        """
        if not os.path.exists(jpg_path):
            warnings.warn(f" > Warning: JPG file not found {jpg_path}. Skipping.")
            return None
            
        # *** 9. Fixed: Use [0] to get correct file basename ***
        npy_path = os.path.splitext(jpg_path)[0] + ".npy"
        
        if os.path.exists(npy_path):
            return npy_path  # Skip existing files
            
        try:
            with Image.open(jpg_path) as img:
                # *** 10. New: Resize step ***
                img_resized = img.resize((IMAGE_SIZE, IMAGE_SIZE), resample=LANCZOS_RESAMPLE)
                
                # Convert to NumPy array
                if img_resized.mode == 'RGBA':
                    img_resized = img_resized.convert('RGB')
                    
                img_array = np.array(img_resized)
                np.save(npy_path, img_array)
                return npy_path
        except Exception as e:
            warnings.warn(f" > Warning: Error processing {jpg_path}: {e}. Skipping.")
            return None

    non_feature_cols.extend(['npy_path'])
    
    tqdm.pandas(desc=" > Converting training set images")
    train_df['npy_path'] = train_df['absolute_image_path'].progress_apply(convert_jpg_to_npy)
    
    tqdm.pandas(desc=" > Converting validation set images")
    val_df['npy_path'] = val_df['absolute_image_path'].progress_apply(convert_jpg_to_npy)
    
    tqdm.pandas(desc=" > Converting test set images")
    test_df['npy_path'] = test_df['absolute_image_path'].progress_apply(convert_jpg_to_npy)

    # Clean again: discard failed conversions
    train_df = train_df.dropna(subset=['npy_path'])
    val_df = val_df.dropna(subset=['npy_path'])
    test_df = test_df.dropna(subset=['npy_path'])
    
    print(" > Image conversion completed.")

    # --- 10, 11, 12. 💾 Save output ---
    print("[Final] Saving all processed files...")

    final_feature_columns = final_continuous_cols + final_categorical_cols
    
    data_splits = {
        'train': train_df,
        'val': val_df,
        'test': test_df
    }
    
    for split_name, df in data_splits.items():
        
        # (Step 10) Save tabular features
        features_path = os.path.join(output_dir, f"{split_name}_features.csv")
        df[final_feature_columns].to_csv(features_path, index=False, header=False)
        
        # (Step 11) Save labels
        labels_path = os.path.join(output_dir, f"{split_name}_labels.pt")
        labels_tensor = torch.tensor(df[label_column].values, dtype=torch.int64)
        torch.save(labels_tensor, labels_path)
        
        # (Step 12) Save image paths
        paths_path = os.path.join(output_dir, f"{split_name}_paths.pt")
        npy_path_list = df['npy_path'].tolist()
        torch.save(npy_path_list, paths_path)

    # (Step 10) Save field lengths
    lengths_path = os.path.join(output_dir, "tabular_lengths.pt")
    torch.save(field_lengths, lengths_path)

    print("-" * 30)
    print("🎉 Preprocessing completed! 🎉")
    print(f"Output files saved to: {output_dir}")
    print(f" > Label: {label_column}")
    print(f" > Tabular features: {len(final_feature_columns)} other attributes")
    print(f" > Split ratio: 8:1:1 (stratified)")
    print(f" > Image processing: Resized to {IMAGE_SIZE}x{IMAGE_SIZE} and converted to .npy")
    print("-" * 30)


if __name__ == "__main__":
    # Run main preprocessing function
    preprocess_celeba_flow(
        attr_file=ATTR_FILE,
        base_image_dir=BASE_IMAGE_DIR,
        output_dir=OUTPUT_DIR,
        label_column=LABEL_COLUMN
    )

开始执行 CelebA 预处理流程 (标签: Attractive)...
 正在加载标注文件 (list_attr_celeba.csv)...
 > 成功加载 202599 条记录。
 正在定义特征...
 > 已定义 0 个连续特征。
 > 已定义 40 个类别特征 (包含标签)。
 > 目标标签 (Label): Attractive
 正在按 8:1:1 对 Attractive 进行分层划分...
 > 训练集: 162079 样本
 > 验证集: 20260 样本
 > 测试集: 20260 样本
 正在筛选恒定特征...
 > 已将 'Attractive' 隔离为标签, 不作为特征。
 > 剩余 0 个连续特征。
 > 剩余 39 个类别特征 (表格属性)。
 正在转换图像路径为绝对路径...
 正在过滤无效行 (基于路径和标签)...
 > 训练集移除 0 行
 > 验证集移除 0 行
 > 测试集移除 0 行
 正在处理缺失值/特殊编码...
 > [CelebA 特定] 转换属性：将 -1 映射到 0...
 正在处理类别特征...
 > CelebA 属性已为 0/1 编码，跳过.cat.codes。
 > 正在计算基数 (Cardinalities)...


/tmp/ipykernel_292101/4247396634.py:196: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if "MISSING" in all_values.unique():


 > 39 个类别特征的基数 (部分): [2, 2, 2, 2, 2]...
 正在标准化连续特征...
 > 没有连续特征，跳过标准化。
 正在转换图像 (JPG -> NPY, 并 Resize 到 224x224)...


 > 转换 测试集 图像: 100%|██████████| 20260/20260 [00:00<00:00, 55325.04it/s]


 > 图像转换完成。
[Final] 正在保存所有处理后的文件...
------------------------------
🎉 预处理全部完成！ 🎉
输出文件已保存到: ./features
 > 标签 (Label): Attractive
 > 表格特征: 39 个其他属性
 > 划分比例: 8:1:1 (分层)
 > 图像处理: 已缩放至 224x224 并转为 .npy
------------------------------


In [ ]:
import pandas as pd
import numpy as np
import sys

# ================================================================
# 1. Please replace this with your CelebA .csv file path
# ================================================================
# Example:
# path_to_your_csv = '/path/to/your/celeba_tabular_data.csv'
path_to_your_csv = '/mnt/hdd/jiazy/CelebA/list_attr_celeba.csv'
# ================================================================

# Replace 'your_file.csv' with your filename
df = pd.read_csv(path_to_your_csv)
has_nan = df.isnull().values.any()
print(f"Does the file contain NaN values: {has_nan}")

文件中是否存在NaN值: False
